In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("year_map.csv")

In [2]:
import json
from pandas.io.json import json_normalize


with open("year_map_10.json", encoding='utf-8') as json_data:
    geometry_data = json.load(json_data)
    
for feat in geometry_data['features']:
    del feat["properties"]['wikipedia']
    del feat["properties"]['code_local']
    del feat["properties"]['note']
    del feat["properties"]['hasc_maybe']
    del feat["properties"]['abbrev']
    del feat["properties"]['fips_alt']
    del feat["properties"]['gn_region']
    del feat["properties"]['sub_code']
    del feat["properties"]['gns_region']
    del feat["properties"]['name_alt']
    del feat["properties"]['name_local']
    del feat["properties"]['postal']
    del feat["properties"]['woe_label']
    del feat["properties"]['gn_name']
    del feat["properties"]['gns_name']
    del feat["properties"]['gns_lang']
    del feat["properties"]['gns_adm1']

    del [feat["properties"]['fips'], feat["properties"]['region_sub'],feat["properties"]['region'], feat["properties"]['region_cod'],feat["properties"]['type_en'], feat["properties"]['type']]

    
    


### First get csv in shape

In [3]:
df1 = pd.DataFrame()

In [4]:
def get_count(row):
    list_ = row["name"].split(",")
    count = len(list_)
    return count

In [5]:
df["properties.headline"] = df.apply(get_count, axis=1)

In [6]:
df["properties.name"] = df["location_english"]

In [7]:
def add_p_tag(row):
    list_ = row["name"].split(",")
    names_with_p_l = ["<p>" + name + "</p>" for name in list_]
    names_with_p = "".join(names_with_p_l)
    return names_with_p

In [8]:
df["properties.article"] = df.apply(add_p_tag, axis=1)

In [9]:
df["properties.year"] = df["year"]

In [10]:
## If Xi Jinping is in a province, style the polygon differently
def get_color(row):
    if "习近平" in row["name"]:
        color = "black"
    else:
        color = "white"
    return color

In [11]:
df["properties.color"] = df.apply(get_color, axis=1)

In [12]:
def get_dash_array(row):
    if "习近平" in row["name"]:
        dash = ""
    else:
        dash = "3"
    return dash

In [13]:
df["properties.dashArray"] = df.apply(get_dash_array, axis=1)

In [14]:
def get_weight(row):
    if "习近平" in row["name"]:
        weight = 4
    else:
        weight = 2
    return weight

In [15]:
df["properties.weight"] = df.apply(get_weight, axis=1)

In [16]:
df1 = df.drop(columns = ["year", "location", "location_lat", "location_lng", "location_english", "name"])

In [17]:
def get_group_name(row):
    group_name = row["properties.year"][:4]
    return group_name

In [18]:
df1["properties.group_name"] = df1.apply(get_group_name, axis=1)

In [19]:
# 1993 --> group 0
# 1994 --> group 1
# 1995 --> group 2
# etc

def get_group_id(row):
    group_id = int(row["properties.year"][:4]) - 1993
    return group_id

In [20]:
df1["properties.group_id"] = df1.apply(get_group_id, axis=1)

In [21]:
df1.to_csv("to_join_shape")

### Now get geojson in shape

In [22]:
out_geometry = pd.DataFrame.from_dict(json_normalize(geometry_data["features"]), orient='columns')

In [23]:
output_complete_test = pd.merge(df1, out_geometry[['geometry.coordinates','geometry.type','properties.name']], left_on='properties.name', right_on='properties.name', how='left')

In [24]:
output_complete = output_complete_test[(output_complete_test["properties.group_id"] >= 0) & (output_complete_test["properties.group_id"] <= 20)]

In [25]:
output_complete.dropna(subset = ["geometry.coordinates"], inplace = True)

C:\Users\Weihua\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
output_complete.to_csv("Geojson_output.csv", index = False)

### df to json

In [27]:
 ok_json = json.loads(output_complete.to_json(orient='records'))

In [28]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [29]:
geo_format = process_to_geojson(ok_json)

In [30]:
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)